# Лабораторная работа №3
## Выполнил студент группы БСТ1904 Костин Антон Алексеевич
### Задание №1

Реализовать методы поиска подстроки в строке. Добавить возможность
ввода строки и подстроки с клавиатуры. Предусмотреть возможность
существования пробела. Реализовать возможность выбора опции
чувствительности или нечувствительности к регистру. Оценить время работы
каждого алгоритма поиска и сравнить его со временем работы стандартной
функции поиска, используемой в выбранном языке программирования. 
#### Алгоритмы: 
1. Кнута-Морриса-Пратта  
2. Упрощенный Бойера-Мура

#### Алгоритм  Кнута-Морриса-Пратта

In [ ]:
# Префикс функция
def prefix(str):
    pi = [0] * len(str)
    i, j = 0, 1
    while j < len(str):
        if str[i] == str[j]:
            pi[j] = i + 1
            i += 1
            j += 1
        elif i:
            i = pi[i - 1]
        else:
            pi[j] = 0
            j += 1
    return pi

# Сам алгоритм
def KMP(str, word, ignore_space=False, ignore_case=False):
    s = str
    w = word

    if ignore_case:
        s = str.lower()
        w = word.lower()

    if ignore_space:
        s = s.replace(' ', '')
        w = w.replace(' ', '')
        
    k = 0;
    A = []
    pi = prefix(str)
    
    for i in range(len(str)):
        while k > 0 and str[i] != word[k]:
            k = pi[k - 1]
        if str[i] == word[k]:
            k += 1
        if k == len(word):
            A.append((i - len(word) + 1, i + 1))
            k = pi[k - 1]
    return A

#### Упрощённый алгоритм Бойера-Мура

In [ ]:
def table(word):
    tab = [len(word)] * 256
    for i in range(len(word) - 1):
        tab[ord(word[i])] = len(word) - 1 - i
    return tab

def BM(str, word, ignore_case=False, ignore_space=False):
    s = str
    w = word

    if ignore_case:
        s = str.lower()
        w = word.lower()

    if ignore_space:
        s = s.replace(' ', '')
        w = w.replace(' ', '')

    A = []
    tab = table(w)
    skip = 0
    while len(s) - skip >= len(w):
        if s[skip:skip + len(w)] == w:
            A.append((skip, skip + len(w)))
        skip += tab[ord(s[skip + len(w) - 1])]
    return A

### Задание 2 «Пятнашки» 
Игра в 15, пятнашки, такен — популярная головоломка, придуманная в 1878 году Ноем Чепмэном. Она представляет собой набор одинаковых квадратных костяшек с нанесёнными числами, заключённых в квадратную коробку. Длина стороны коробки в четыре раза больше длины стороны костяшек для набора из 15 элементов, соответственно в коробке остаётся незаполненным одно квадратное поле. Цель игры — перемещая костяшки по коробке, добиться упорядочивания их по номерам, желательно сделав как можно меньше перемещений.   

<b>Задача:</b> написать программу, определяющую, является ли данное расположение «решаемым», то есть можно ли из него за конечное число шагов перейти к правильному. Если это возможно, то необходимо найти хотя бы одно решение - последовательность движений, после которой числа будут расположены в правильном порядке.  

<b>Входные данные:</b> массив чисел, представляющий собой расстановку в порядке «слева направо, сверху вниз». Число 0 обозначает пустое поле.  
Например, массив [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0] представляет собой «решенную» позицию элементов. 

<b>Выходные данные:</b> если решения нет, то функция должна вернуть пустой массив []. Если решение есть, то необходимо представить решение — для каждого шага записывается номер передвигаемого на данном шаге элемента.

<b>Например,</b> для начального расположения элементов [1, 2, 3, 4, 5, 6, 7, 8, 13, 9, 11, 12, 10, 14, 15, 0] одним из возможных решений будет  
[15, 14, 10, 13, 9, 10, 14, 15] (последовательность шагов здесь: двигаем 15, двигаем 14, двигаем 10, …, двигаем 15)

In [3]:
from queue import PriorityQueue

class Position:
    def __init__(self, position, start_distance):
        self.position = position
        self.start_distance = start_distance

    def __lt__(self, other):
        return self.start_distance < other.start_distance

    def __str__(self):
        return '\n'.join((N*'{:3}').format(*[i%(N*N) for i in self.position[i:]]) for i in range(0, N*N, N))

N = 4

def moves(position):
    blank = position.index(0)
    i, j = divmod(blank, N)
    offsets = []
    if i > 0: offsets.append(-N)     # вниз
    if i < N - 1: offsets.append(N)  # вверх
    if j > 0: offsets.append(-1)     # вправо
    if j < N - 1: offsets.append(1)  # влево
    for offset in offsets:
        swap = blank + offset
        yield tuple(position[swap] if x==blank else position[blank] if x==swap else position[x] for x in range(N*N))


def parity(permutation):
    seen, cycles = set(), 0
    for i in permutation:
        if i not in seen:
            cycles += 1
            while i not in seen:
                seen.add(i)
                i = permutation[i]
    return (cycles + len(permutation)) % 2




SOLVED = (1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0)
start = [1, 2, 3, 4, 5, 6, 7, 8, 13, 9, 11, 12, 10, 14, 15, 0]

if parity(start) == 0:
    print('Нерешаемо')
else:
    start = tuple(start)
    p = Position(start, 0)
    print(p, '\n')

    candidates = PriorityQueue()
    candidates.put(p)

    visited = set([p])
    came_from = {p.position: None}
    
    while p.position != SOLVED:
        p = candidates.get()
        for k in moves(p.position):
            if k not in visited:
                candidates.put(Position(k, p.start_distance + 1))
                came_from[k] = p
                visited.add(k)

    path = []
    prev = p
    while p.position != start:
        p = came_from[p.position]
        number = p.position[prev.position.index(0)]
        path.append(number)
        prev = p
    path.reverse()

    print(path)

  1  2  3  4
  5  6  7  8
 13  9 11 12
 10 14 15  0 


[15, 14, 10, 13, 9, 10, 14, 15]


### Вывод
